In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [37]:
#The original data has been named train_original
#Here I am splitting the data into a train and test set. I want to stratify by age. 
# Then I export as csv files, since we are working over multiple jupiter notebooks.

from sklearn.model_selection import train_test_split

train_original=pd.read_csv('train_original.csv')

seed=1275
train, test = train_test_split(train_original, test_size=0.2, random_state=seed, stratify=train_original['Basic_Demos-Age'])

train.to_csv('train.csv', index=False)
test.to_csv('test.csv',index=False)

In [68]:
#This is the starting data.
train=pd.read_csv('train.csv')

**Adding Accelerometer Data**

We have accelerometer data that should be merged into this data set

Note that there are some participants who appear to have accelerometer data but aren't listed in train (they're likely in test). So we'll need to do a 'left' join to avoid incorporating participants who aren't in train

In [69]:
# Load the accelerometer data set Accelerometer_enmo_anglez_daily_averages.csv
accel = pd.read_csv('Accelerometer_enmo_anglez_daily_averages.csv')

# Join train and accel  on the 'id' column and accel on the 'ID' column
train = train.join(accel.set_index('ID'), on='id', how='left')

# It seems unlikly that we're going to want the ENMO_Avg_All_Days_MVPA192 or ENMO_Avg_All_Days_MVPA110 or Positive_Anglez_All_Days variables, so remove them
train = train.drop(columns=['ENMO_Avg_All_Days_MVPA192', 'ENMO_Avg_All_Days_MVPA110', 'Positive_Anglez_All_Days']) 

**Averaging Sit & Reach Data**

The Sit & Reach test is done twice, once with the left leg extended (SRL) and once with the right leg extended (SRR). Measuring this twice seems redundant, so we'll create a new variable that is the average of the right & left variables, then delete the SRL and SRR variables.

In [70]:
# Create a new variable 'FGC-FGC_SR' that is the mean of FGC-FGC_SRL and FGC-FGC_SRR
train['FGC-FGC_SR'] = train[['FGC-FGC_SRL', 'FGC-FGC_SRR']].mean(axis=1)

# Remove the old sit & reach variables
train = train.drop(columns=['FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone'])

**Calculating Sit & Reach Zone**

FitnessGram Healthy Fitness Zones are documented at https://pftdata.org/files/hfz-standards.pdf

We can use these to compute a new Zone variable for sit & reach

Note that Basic_Demos-Sex is coded as 0=Male and 1=Female

In [71]:
# Create a new variable 'FGC-FGC_SR_Zone' that is equal to 1 if any of the following are true:
# Basic_Demos-Sex==0 and FGC-FGC_SR >= 8
# Basic_Demos-Sex==1 and FGC-FGC_SR_Zone >= 9 and Basic_Demos-Age is between 5 and 10
# Basic_Demos-Sex==1 and FGC-FGC_SR_Zone >= 10 and Basic_Demos-Age is between 11 and 14
# Basic_Demos-Sex==1 and FGC-FGC_SR_Zone >= 12 and Basic_Demos-Age is at least 15

# One way to do this is to define a function that would take sex, age, and SR value as inputs and output 1 or 0
def sitreachzone(sex:int, age:int, sr:int):
    if sex == 0:
        if sr >= 8:
            return 1
        elif sr <= 7:
            return 0
        else:
            return np.nan
    elif sex == 1:
        if age >= 15:
            if sr >= 12:
                return 1
            elif sr <= 11:
                return 0
            else:
                return np.nan
        elif age >= 11:
            if sr >= 10:
                return 1
            elif sr <=9:
                return 0
            else:
                return np.nan
        else:
            if sr >= 9:
                return 1
            elif sr <= 8:
                return 0
            else:
                return np.nan
    else:
        return np.nan

# Apply sitreachzone to create a new column using the columns Basic_Demos-Sex, Basic_Demos-Age, and FGC-FGC_SR as inputs
train['FGC-FGC_SR_Zone'] = train.apply(lambda x: sitreachzone(x['Basic_Demos-Sex'], x['Basic_Demos-Age'], x['FGC-FGC_SR']), axis=1)


# Note: The internet suggests that using .loc is vectorized, so much faster than using .apply. Below is a faster version that we could use if necessary
#train['FGC-FGC_SR_Zone'] = train.loc[(train['Basic_Demos-Sex']==0) & (train['FGC-FGC_SR'] >= 8)] = 1
#train['FGC-FGC_SR_Zone'] = train.loc[(train['Basic_Demos-Sex']==1) & (train['FGC-FGC_SR'] >= 12)] = 1
#train['FGC-FGC_SR_Zone'] = train.loc[(train['Basic_Demos-Sex']==1) & (train['FGC-FGC_SR'] >= 10) & (train['FGC-FGC_SR'] <= 14)] = 1
#train['FGC-FGC_SR_Zone'] = train.loc[(train['Basic_Demos-Sex']==1) & (train['FGC-FGC_SR'] >= 9) & (train['FGC-FGC_SR'] <= 10)] = 1
#train['FGC-FGC_SR_Zone'] = train['FGC-FGC_SR_Zone'].fillna(0)


**Creating the PAQ_MVPA_60 Variable**

Some research (https://pubmed.ncbi.nlm.nih.gov/27759968/) has identified a cut-off score of 2.75 (A) and 2.73 (C) to discriminate >60 minutes of MVPA. However, the study suggests that, while the cutoff is significant for PAQ-A, it isn't for PAQ-C.

With that caveat, before combining PAQ-A and PAQ-C, we'll create a new binary variable that flags for >60 minutes of MVPA, called MVPA_60

In [72]:
# Create a new variable that is 1 when PAQA/C Total is at least 2.75/2.73, 0 if it's less than these cutoffs, and NaN if PAQA/C is NaN
train['PAQA_MVPA'] = np.where(train['PAQ_A-PAQ_A_Total']>=2.75, 1, 0)
train['PAQA_MVPA'] = np.where(train['PAQ_A-PAQ_A_Total'].isnull(), np.nan, train['PAQA_MVPA'])

train['PAQC_MVPA'] = np.where(train['PAQ_C-PAQ_C_Total']>=2.73, 1, 0)
train['PAQC_MVPA'] = np.where(train['PAQ_C-PAQ_C_Total'].isnull(), np.nan, train['PAQC_MVPA'])

**Combining PAQ_A and PAQ_C Predictors**

The variables PAQ_A (Season and Total) and PAQ_C (Season and Total) both report "Information about children's participation in vigorous activities over the last 7 days." 
* More information about PAQ-C is available here: https://fcon_1000.projects.nitrc.org/indi/cmi_healthy_brain_network_old/assessments/paq-c.html. It is administered to participnats age 8-14
* More information about PAQ-A is available here: https://fcon_1000.projects.nitrc.org/indi/cmi_healthy_brain_network_old/assessments/paq-a.html. It is administered to participants age 14-19

These scores appear to be comparable, so we can combine them. However, prior to doing so, we should note that there could be participants who had scores for both measures. This would occur if their age was recorded in a different season than the PCIAT and/or PAQ seasons.

If we were being really careful, we'd construct a complicated function to account for these cases. However, exploration of the original training set (3600 participants) only found one such case, so it seems like it might be a relatively rare event. In addition, by combining the two PAQ columns we are assuming that the two scores are comparable. Thus, it makes sense to keep either of the A/C values.

For the one subject in the original data, their recorded age was 13; their PCIAT-Season and PAQ_C-Season were Spring, so when we combine these variables we'll keep the PAQ_C values.

In [73]:
# Create new variables that merge the three PAQA/C variables
train['PAQ_Total']=train['PAQ_C-PAQ_C_Total']
train.loc[train['PAQ_Total'].isnull(),'PAQ_Total']=train['PAQ_A-PAQ_A_Total']

train['PAQ_Season']=train['PAQ_C-Season']
train.loc[train['PAQ_Season'].isnull(),'PAQ_Season']=train['PAQ_A-Season']

train['PAQ_MVPA']=train['PAQC_MVPA']
train.loc[train['PAQ_MVPA'].isnull(),'PAQ_MVPA']=train['PAQA_MVPA']

# Drop the PAQ variables we no longer need
train=train.drop(columns=['PAQ_C-PAQ_C_Total', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season', 'PAQ_A-Season', 'PAQA_MVPA', 'PAQC_MVPA'])

**Creating the Fitness Endurance Variable**

There are currently separate variables for fitness endurance minutes & seconds. We'll combine these into a single variable that measures the total number of seconds

In [74]:
# Combine the minutes and seconds of Fitness_Endurance into a single number (total number of seconds)
train['Fitness_Endurance_Total_Time_Sec'] = train['Fitness_Endurance-Time_Mins'] * 60 + train['Fitness_Endurance-Time_Sec']

train=train.drop(columns=['Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec'])

**Sleep Disturbance Scale Cleaning**

The sleep disturbance scale was created/documented in 1996: https://pubmed.ncbi.nlm.nih.gov/9065877/

There are two Sleep Disturbance Scale variables: the "Raw" score and the Total T-Score. 
I can't find much information about what these mean. But if the T-Score is just a standardized version of the Raw score, then they should be conveying identical information.

Their correlation is 0.995731. It seems reasonable and safe to drop one of them.

In [75]:
# Remove the SDS-SDS_Total_T variable from train
train=train.drop(columns=['SDS-SDS_Total_T'])

**BIA Variable Cleaning**

Some of the BIA variables appear to either be redundant or computed from each other:
* BIA-BIA_BMI is measuring BMI. It has correlation 0.965105 with Physical_BMI. I can't find any information about how it is any different. It seems likely that BMI varies from day to day and measurement to measurement, so the difference is likely due to measurement error. Going to remove BIA-BIA_BMI

The other BIA variables are:
* BIA-BIA_BMC	Bone Mineral Content
* BIA-BIA_BMR	Basal Metabolic Rate
* BIA-BIA_DEE	Daily Energy Expenditure
* BIA-BIA_LDM	Lean Dry Mass
* BIA-BIA_LST	Lean Soft Tissue
* BIA-BIA_SMM	Skeletal Muscle Mass
* BIA-BIA_FFM	Fat Free Mass
* BIA-BIA_Fat	Body Fat Percentage
* BIA-BIA_Frame_num	Body Frame
* BIA-BIA_TBW	Total Body Water
* BIA-BIA_ECW	Extracellular Water
* BIA-BIA_ICW	Intracellular Water
* BIA-BIA_FMI	Fat Mass Index: Calcuated as FM divided by height squared
* BIA-BIA_FFMI	Fat Free Mass Index: Calcuated as FFM divided by height squared

There exist large correlations between many pairs of these predictors. Below is a list of all pairs with at least 0.99 correlation:
* FFM	BMR	0.9999999999991445
* BMR	TBW	0.9996843922178612
* FFM	TBW	0.9996843882486972
* ECW	BMR	0.99949744961038
* ECW	FFM	0.9994974489820077
* TBW	ECW	0.9994121539401678
* TBW	ICW	0.9989842717508111
* FFM	LDM	0.9989337841842884
* BMR	LDM	0.9989337768885788
* SMM	ICW	0.9984531398777805
* ICW	BMR	0.9981423016493606
* FFM	ICW	0.998142293296351
* ECW	LDM	0.9980088142821305
* TBW	SMM	0.9976684856970098
* TBW	LDM	0.9974587197051186
* ICW	LST	0.9969014072300802
* SMM	BMR	0.996864975676666
* FFM	SMM	0.9968649649869374
* ECW	ICW	0.996852207975357
* LST	SMM	0.9967543066184706
* SMM	ECW	0.9957140182568774
* TBW	LST	0.995487594282981
* BMC	LDM	0.9950199542105168
* LDM	ICW	0.9949518859314233
* ICW	DEE	0.9948088046663506
* DEE	TBW	0.9944107160263206
* DEE	BMR	0.9941186367579632
* DEE	FFM	0.9941186344807962
* LDM	SMM	0.9937473329914341
* LST	BMR	0.9936088620550899
* LST	FFM	0.99360884373626
* DEE	SMM	0.9933843458863473
* LST	ECW	0.9930574064543182
* ECW	DEE	0.992754802616644
* DEE	LDM	0.9919453675327521
* FFM	BMC	0.9914085178617896
* BMR	BMC	0.9914084966327703
* LST	DEE	0.9912776109118937
* BMC	ECW	0.9909706551293764

If we keep the BIA-BIA_FFM variable, then we could eliminate:
* BMR
* TBW
* ECW
* ICW
* LDM
* SMM
* DEE
* LST
* BMC

In [76]:
# Remove BIA-BIA_BMI from train
train=train.drop(columns=['BIA-BIA_BMI'])

# Remove the following variables from train: BIA-BIA_BMR, BIA-BIA_TBW, BIA-BIA_ECW, BIA-BIA_LDM, BIA-BIA_ICW, BIA-BIA_SMM, BIA-BIA_DEE, BIA-BIA_LST, and BIA-BIA_BMC
train=train.drop(columns=['BIA-BIA_BMR', 'BIA-BIA_TBW', 'BIA-BIA_ECW', 'BIA-BIA_LDM', 'BIA-BIA_ICW', 'BIA-BIA_SMM', 'BIA-BIA_DEE', 'BIA-BIA_LST', 'BIA-BIA_BMC'])

**Removing Nonsensical Values and Outliers**

This section focuses on removing "bad" data. We'll only do this for "Float" variables, so we'll begin by making a list for analysis.

In [77]:
#I only want to include quantitative variables for possible analysis for outliers, so first I identify the float variables.

float_columns = train.select_dtypes(include=['float']).columns
print("Float columns:", float_columns)
print(len(float_columns))

Float columns: Index(['CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight',
       'Physical-Waist_Circumference', 'Physical-Diastolic_BP',
       'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
       'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_FFM', 'BIA-BIA_FFMI',
       'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'PCIAT-PCIAT_01',
       'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05',
       'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09',
       'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13',
       'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17',
       'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PCIAT-PCIAT_Total', 'SDS-SDS_Total_R

In [57]:
# All of the Physical- variables should have non-zero values (this may not be the case for some other variables). 
# Negative values will be taken care of later, but we'll replace the zeroes here:

# For each variable that starts with 'Physical-' replace any values that are 0 with NaN
for column in train.columns:
    if column.startswith('Physical-'):
        train[column] = train[column].replace(0, np.nan)

In [58]:
#This generates a list of indices for extreme outliers or negative values that should not be negative. 
#In particular, BIA_BIA_Fat, which measures body fat percentage, has a bunch of negative values.
threshold=5
index_set=[]
for i in range(len(float_columns)):
    #print(float_columns[i])
    z=np.abs(stats.zscore(train[[float_columns[i]]],nan_policy='omit'))
    indices_large_z = np.where(np.all(z > threshold, axis=1))[0].flatten().tolist()
    indices_negative=np.where(np.all(train[[float_columns[i]]]<0, axis=1))[0].flatten().tolist()
    index_set=index_set+indices_large_z+indices_negative
    #print(indices_large_z)
    #print(indices_negative)
    i+=1

index_set_neg_largez=list(set(index_set))
print(index_set_neg_largez)
print(len(index_set_neg_largez))

[515, 1541, 1542, 3080, 2061, 2078, 1054, 1058, 1067, 3120, 1076, 573, 2116, 1092, 3142, 73, 76, 78, 3150, 1621, 92, 1119, 2656, 610, 2658, 101, 107, 2171, 1663, 127, 1671, 2185, 657, 1683, 2720, 2723, 1188, 1205, 1214, 2249, 1234, 2265, 2276, 1766, 2287, 2799, 2804, 2296, 1274, 1791, 1287, 777, 786, 793, 797, 2853, 1320, 1836, 2862, 2350, 2352, 306, 308, 2870, 2875, 2363, 2366, 2881, 329, 1354, 1867, 2387, 1893, 872, 2412, 371, 1399, 2425, 2954, 2446, 1427, 2966, 411, 1441, 1955, 2486, 962, 3012, 2518, 2525, 992, 997, 3047, 488, 498, 2042]
96


In [59]:
#This defines a new dataframe with the the entire case removed, where there is an extreme outlier or a nonsensical negative value.
train_no_outliers_no_negatives=train.drop(index_set_neg_largez)

#Here's a quick look at the effect of this cleaning. The first entry shows how many rows have been affected in each sii category. 

print(train['sii'].value_counts()-train_no_outliers_no_negatives['sii'].value_counts())

sii
0.0    38
1.0    22
2.0    21
3.0     5
Name: count, dtype: int64


In [60]:
#This takes extreme outliers and nonsensical negative values and replaces them with NaN.

train_outliers_neg_to_NaN=train
for i in range(len(float_columns)):
    index_set=[]
    z=np.abs(stats.zscore(train[[float_columns[i]]],nan_policy='omit'))
    indices_large_z = np.where(np.all(z > threshold, axis=1))[0].flatten().tolist()
    indices_negative=np.where(np.all(train[[float_columns[i]]]<0, axis=1))[0].flatten().tolist()
    index_set=indices_large_z+indices_negative
    #print(len(index_set))
    for j in index_set:
        train_outliers_neg_to_NaN.at[j, float_columns[i]] = np.nan
        j+=1
    i+=1


In [61]:
#Now I replace the original train dataframe with the new cleaned data frame (where I've replaced bad values with NaN.)

train=train_outliers_neg_to_NaN
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 84 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3168 non-null   object 
 1   Basic_Demos-Enroll_Season               3168 non-null   object 
 2   Basic_Demos-Age                         3168 non-null   int64  
 3   Basic_Demos-Sex                         3168 non-null   int64  
 4   CGAS-Season                             2065 non-null   object 
 5   CGAS-CGAS_Score                         1950 non-null   float64
 6   Physical-Season                         2642 non-null   object 
 7   Physical-BMI                            2391 non-null   float64
 8   Physical-Height                         2404 non-null   float64
 9   Physical-Weight                         2396 non-null   float64
 10  Physical-Waist_Circumference            695 non-null    floa

In [ ]:
#I'll export the dataframe to a csv.

train_cleaned=train
train_cleaned.to_csv('train_cleaned.csv', index=False)

Next I am going to impute input variables. 
I'm doing this before I remove the cases for which we can't compute sii scores, so that we have all data available.
I am doing this in groups: For example, I will use only physical data to impute physical data values. This seems reasonable to do, although perhaps we might get more accurate results if we used more variables?

In [91]:
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#Because we will be using multiple imputation strategies, 
# I am going to define a new dataframe that will record all of the imputations using KNN.

train_imp_KNN=train_cleaned.copy()

# define a pipe that first scales the variables and then does a KNN imputation. 
# Note that when there is a case with no values at all, KNNImputer replaces fills in each variable with the group average.

Number_Neighbors=5
impute_pipe = Pipeline([('scale', StandardScaler()),
                 ('KNN_impute', KNNImputer(n_neighbors=Number_Neighbors, weights='uniform', metric='nan_euclidean'))])




In [92]:
#We have complete information for the basic demographics variables, age and gender.

Basic_Demos = [col for col in train_imp_KNN.columns if 'Basic' in col]
Basic_Demos.remove('Basic_Demos-Enroll_Season')
train_imp_KNN['Basic_nan_count'] = train_imp_KNN[Basic_Demos].isna().sum(axis=1)
train_imp_KNN['Basic_nan_count'].value_counts()

Basic_nan_count
0    3168
Name: count, dtype: int64

In [93]:
#Next we'll consider the physical variables. There are many missing values here, including 688 cases with no values at all. We will do imputation.
#Because age and gender are likely to be related to the Physical variables, I add these to the mix for imputation.
#Note also that I have removed the season variable. I did this because it is not quantitative, so I can't easily run the imputation using this variable. 
#But this might be something to go back to later.

Physical = [col for col in train_imp_KNN.columns if 'Physical' in col]
Physical.remove('Physical-Season')
Physical=Physical+Basic_Demos
train_imp_KNN['Physical_nan_count'] = train_imp_KNN[Physical].isna().sum(axis=1)
print(train_imp_KNN['Physical_nan_count'].value_counts())
print(len(Physical))

Physical_nan_count
1    1663
7     716
0     664
6      45
4      30
3      27
2      22
5       1
Name: count, dtype: int64
9


In [94]:
#Now I will impute values for these variables. First I'll define a new dataframe to work on.

df=train_imp_KNN[Physical]

#Now I run the impute pipe on this dataframe. First I fit the pipe to the data. I record the transform of the dataframe as imputation. 
# Imputation is a numpy array, so it needs to be converted back to a pandas dataframe.
#Also, I reverse-transformed the data. My reasoning for doing this is that we want it in terms of the original scale to be able to make sense of things. 
#But since we are scaling twice, more rounding issues arise.

impute_pipe.fit(df)

imputation_physical=impute_pipe.transform(df)
imputation_physical=impute_pipe.named_steps['scale'].inverse_transform(imputation_physical)
df2 = pd.DataFrame(imputation_physical, columns=Physical)
df2.info()

#Lastly, I replace the original values in the dataframe with the newly imputed values.

train_imp_KNN[Physical]=train_imp_KNN[Physical].fillna(df2[Physical])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Physical-BMI                  3168 non-null   float64
 1   Physical-Height               3168 non-null   float64
 2   Physical-Weight               3168 non-null   float64
 3   Physical-Waist_Circumference  3168 non-null   float64
 4   Physical-Diastolic_BP         3168 non-null   float64
 5   Physical-HeartRate            3168 non-null   float64
 6   Physical-Systolic_BP          3168 non-null   float64
 7   Basic_Demos-Age               3168 non-null   float64
 8   Basic_Demos-Sex               3168 non-null   float64
dtypes: float64(9)
memory usage: 222.9 KB


In [95]:
#Next we'll consider the fitness test variables. 
# There are many missing values here, although it looks like we have at least some values for every case.
#I kept in all of the zone variables, which means they have the same weight as the actual measurements. It seems like I shouldn't do this.

Fitness = [col for col in train_imp_KNN.columns if 'Fitness' in col]+[col for col in train_imp_KNN.columns if 'FGC' in col]
Fitness.remove('Fitness_Endurance-Season')
Fitness.remove('FGC-Season')
Fitness=Fitness+Basic_Demos
train_imp_KNN['Fitness_nan_count'] = train_imp_KNN[Fitness].isna().sum(axis=1)
print(train_imp_KNN['Fitness_nan_count'].value_counts())
print(len(Fitness))

Fitness_nan_count
17    1306
3      649
7      549
4      433
0      152
5       24
9       18
8       10
11       7
12       6
2        5
14       3
6        2
13       2
16       1
15       1
Name: count, dtype: int64
19


In [96]:
#Now I will impute values for these variables. First I'll define a new dataframe to work on.

df=train_imp_KNN[Fitness]

#Now I run the impute pipe on this dataframe. First I fit the pipe to the data. I record the transform of the dataframe as imputation. 
# Imputation is a numpy array, so it needs to be converted back to a pandas dataframe.

impute_pipe.fit(df)

imputation_fitness=impute_pipe.transform(df)
imputation_fitness=impute_pipe.named_steps['scale'].inverse_transform(imputation_fitness)
df2 = pd.DataFrame(imputation_fitness, columns=Fitness)
df2.info()

#Lastly, I replace the original values in the dataframe with the newly imputed values.

train_imp_KNN[Fitness]=train_imp_KNN[Fitness].fillna(df2[Fitness])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fitness_Endurance-Max_Stage  3168 non-null   float64
 1   Fitness_Endurance-Time_Mins  3168 non-null   float64
 2   Fitness_Endurance-Time_Sec   3168 non-null   float64
 3   FGC-FGC_CU                   3168 non-null   float64
 4   FGC-FGC_CU_Zone              3168 non-null   float64
 5   FGC-FGC_GSND                 3168 non-null   float64
 6   FGC-FGC_GSND_Zone            3168 non-null   float64
 7   FGC-FGC_GSD                  3168 non-null   float64
 8   FGC-FGC_GSD_Zone             3168 non-null   float64
 9   FGC-FGC_PU                   3168 non-null   float64
 10  FGC-FGC_PU_Zone              3168 non-null   float64
 11  FGC-FGC_SRL                  3168 non-null   float64
 12  FGC-FGC_SRL_Zone             3168 non-null   float64
 13  FGC-FGC_SRR       

In [97]:
#Next we'll consider the BIA variables. 

BIA = [col for col in train_imp_KNN.columns if 'BIA' in col]
BIA.remove('BIA-Season')
BIA=BIA+Basic_Demos
train_imp_KNN['BIA_nan_count'] = train_imp_KNN[BIA].isna().sum(axis=1)
print(train_imp_KNN['BIA_nan_count'].value_counts())
print(len(BIA))


BIA_nan_count
16    1575
0     1527
1       30
2       21
12       7
3        5
4        2
5        1
Name: count, dtype: int64
18


In [98]:
#Now I will impute values for these variables. First I'll define a new dataframe to work on.

df=train_imp_KNN[BIA]

#Now I run the impute pipe on this dataframe. First I fit the pipe to the data. I record the transform of the dataframe as imputation. 
# Imputation is a numpy array, so it needs to be converted back to a pandas dataframe.

impute_pipe.fit(df)

imputation_BIA=impute_pipe.transform(df)
imputation_BIA=impute_pipe.named_steps['scale'].inverse_transform(imputation_BIA)
df2 = pd.DataFrame(imputation_BIA, columns=BIA)
df2.info()

#Lastly, I replace the original values in the dataframe with the newly imputed values.

train_imp_KNN[BIA]=train_imp_KNN[BIA].fillna(df2[BIA])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BIA-BIA_Activity_Level_num  3168 non-null   float64
 1   BIA-BIA_BMC                 3168 non-null   float64
 2   BIA-BIA_BMI                 3168 non-null   float64
 3   BIA-BIA_BMR                 3168 non-null   float64
 4   BIA-BIA_DEE                 3168 non-null   float64
 5   BIA-BIA_ECW                 3168 non-null   float64
 6   BIA-BIA_FFM                 3168 non-null   float64
 7   BIA-BIA_FFMI                3168 non-null   float64
 8   BIA-BIA_FMI                 3168 non-null   float64
 9   BIA-BIA_Fat                 3168 non-null   float64
 10  BIA-BIA_Frame_num           3168 non-null   float64
 11  BIA-BIA_ICW                 3168 non-null   float64
 12  BIA-BIA_LDM                 3168 non-null   float64
 13  BIA-BIA_LST                 3168 

In [99]:
#Next we consider CGAS (Children's Global Assessment Score). This measure comes from an evaluation by a trained professional. 
#Looking at the description, it seems reasonable that it is related to gender and age, so I am going to do KNN with those variables. 

CGAS = ['CGAS-CGAS_Score']+Basic_Demos
train_imp_KNN['CGAS_nan_count'] = train_imp_KNN[CGAS].isna().sum(axis=1)
print(train_imp_KNN['CGAS_nan_count'].value_counts())
print(len(CGAS))
print(CGAS)

CGAS_nan_count
0    1950
1    1218
Name: count, dtype: int64
3
['CGAS-CGAS_Score', 'Basic_Demos-Age', 'Basic_Demos-Sex']


In [100]:
#Now I will impute values for this variable. First I'll define a new dataframe to work on.

df=train_imp_KNN[CGAS]

#Now I run the impute pipe on this dataframe. First I fit the pipe to the data. I record the transform of the dataframe as imputation. 
# Imputation is a numpy array, so it needs to be converted back to a pandas dataframe.

impute_pipe.fit(df)

imputation_CGAS=impute_pipe.transform(df)
imputation_CGAS=impute_pipe.named_steps['scale'].inverse_transform(imputation_CGAS)
df2 = pd.DataFrame(imputation_CGAS, columns=CGAS)
df2.info()

#Lastly, I replace the original values in the dataframe with the newly imputed values.

train_imp_KNN[CGAS]=train_imp_KNN[CGAS].fillna(df2[CGAS])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CGAS-CGAS_Score  3168 non-null   float64
 1   Basic_Demos-Age  3168 non-null   float64
 2   Basic_Demos-Sex  3168 non-null   float64
dtypes: float64(3)
memory usage: 74.4 KB


In [101]:
#Next we consider  PreInt_EduHx-computerinternet_hoursday. 
#It seems reasonable that it is related to gender and age, so I am going to do KNN with those variables. 

IntHrs = ['PreInt_EduHx-computerinternet_hoursday']+Basic_Demos
train_imp_KNN['IntHrs_nan_count'] = train_imp_KNN[IntHrs].isna().sum(axis=1)
print(train_imp_KNN['IntHrs_nan_count'].value_counts())
print(len(IntHrs))


IntHrs_nan_count
0    2633
1     535
Name: count, dtype: int64
3


In [102]:
#Now I will impute values for this variable. First I'll define a new dataframe to work on.

df=train_imp_KNN[IntHrs]

#Now I run the impute pipe on this dataframe. First I fit the pipe to the data. I record the transform of the dataframe as imputation. 
# Imputation is a numpy array, so it needs to be converted back to a pandas dataframe.

impute_pipe.fit(df)

imputation_IntHrs=impute_pipe.transform(df)
imputation_IntHrs=impute_pipe.named_steps['scale'].inverse_transform(imputation_IntHrs)
df2 = pd.DataFrame(imputation_IntHrs, columns=IntHrs)
df2.info()

#Lastly, I replace the original values in the dataframe with the newly imputed values.

train_imp_KNN[IntHrs]=train_imp_KNN[IntHrs].fillna(df2[IntHrs])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 3 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   PreInt_EduHx-computerinternet_hoursday  3168 non-null   float64
 1   Basic_Demos-Age                         3168 non-null   float64
 2   Basic_Demos-Sex                         3168 non-null   float64
dtypes: float64(3)
memory usage: 74.4 KB


This section of the file focus on imputing values for the PCIAT questionaire, based on other responses to the questionaire, using k nearest neighbors.

In [103]:
#First we identify the columns of interest to search for NaN variables.

pciats = [col for col in train_imp_KNN.columns if 'PCIAT' in col]
pciats.remove('PCIAT-Season')
pciats.remove('PCIAT-PCIAT_Total')

# Create 20 new variables that indicate whether or not PCIAT-PCIAT_01 through PCIAT-PCIAT_20 were NaN

for pciat in pciats:
    train_imp_KNN[pciat + '_isnotna'] = train_imp_KNN[pciat].notna().astype(int)

    

#Create new variable that represents the sum of the questions answered.

pciatsnotna = [col for col in train_imp_KNN.columns if 'isnotna' in col]
train_imp_KNN['pciatsnotna_sum'] = train_imp_KNN[pciatsnotna].sum(axis=1)

#Here, we can see NaN rates. Note that there are almost 1000 cases where the participants did not respond to any PCIAT questions. 
#These participants are eliminated. Note that when rows are eliminated, the original row indexing is preserved; therefore, I reset the index. 
#(This step is important to make sure indices match up when I later replace NaN entries with their imputed values.)

print("NaN rates in original data: ",train_imp_KNN['pciatsnotna_sum'].value_counts())
train_imp_KNN = train_imp_KNN[train_imp_KNN['pciatsnotna_sum'] != 0]
train_imp_KNN.reset_index(drop=True, inplace=True)
print("NaN rates after the non-responders have been removed:",train_imp_KNN['pciatsnotna_sum'].value_counts())



NaN rates in original data:  pciatsnotna_sum
20    2130
0      974
19      52
18       9
10       1
15       1
17       1
Name: count, dtype: int64
NaN rates after the non-responders have been removed: pciatsnotna_sum
20    2130
19      52
18       9
10       1
15       1
17       1
Name: count, dtype: int64


In [104]:
#Next we use KNN to impute the missing values.


# define imputer
Number_Neighbors=5
imputer = KNNImputer(n_neighbors=Number_Neighbors, weights='uniform', metric='nan_euclidean')

#The imputer.fit_transform function outputs a numpy array. So first I do the fitting, then convert the output back to a pandas dataframe.

imputations=imputer.fit_transform(train_imp_KNN[pciats])
df2 = pd.DataFrame(imputations, columns=pciats)

#Next take the result and insert into the original dataframe. 

train_imp_KNN[pciats]=train_imp_KNN[pciats].fillna(df2[pciats])

In [105]:
#I recalculate the PCIAT total score. I also drop the variables that were added to detect NaN in the PCIAT data, just to tidy up a bit.

train_imp_KNN['PCIAT_Total_Imputed'] = train_imp_KNN[pciats].sum(axis=1)
train_imp_KNN = train_imp_KNN.drop(columns=pciatsnotna)
train_imp_KNN = train_imp_KNN.drop(columns=['pciatsnotna_sum'])


#Now we can calculate a new sii score with the imputed values. 

bins = [0, 30, 49,79,100]
labels = [0,1,2,3]
train_imp_KNN['sii_Imputed'] = pd.cut(train_imp_KNN['PCIAT_Total_Imputed'], bins=bins, labels=labels, right=False)

print(train_imp_KNN.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2194 entries, 0 to 2193
Data columns (total 90 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   id                                      2194 non-null   object  
 1   Basic_Demos-Enroll_Season               2194 non-null   object  
 2   Basic_Demos-Age                         2194 non-null   int64   
 3   Basic_Demos-Sex                         2194 non-null   int64   
 4   CGAS-Season                             1893 non-null   object  
 5   CGAS-CGAS_Score                         2194 non-null   float64 
 6   Physical-Season                         2074 non-null   object  
 7   Physical-BMI                            2194 non-null   float64 
 8   Physical-Height                         2194 non-null   float64 
 9   Physical-Weight                         2194 non-null   float64 
 10  Physical-Waist_Circumference            2194 non

In [106]:
train_imp_KNN.to_csv('train_imp_KNN.csv', index=False)

Next I'm working on imputation using MICE. As you'll see below, I will use MICE to impute the missing input values, provided the variables are quantitative or categorical binary or ordinal (which are treated as integers in the data). This doesn't necessarily seem like the best idea for the ordinal data. Also, imputation can't directly handle non-ordinal categorical variables with more than 2 categories. In this case, the issue only affects seasons. I could work around this, but for now I just ignored these variables.

In [107]:
# I am going to define a new dataframe that will record all of the imputations using MICE. I only want to apply MICE to the input variables, so I separate those out.
#Also, MICE doesn't like categorical variables. I have just removed those--the seasons--for now.

train_imp_MICE=train_cleaned.copy()

print(train_imp_MICE.columns)

features=['Basic_Demos-Age', 'Basic_Demos-Sex',
        'CGAS-CGAS_Score', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
        'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
        'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 
       'PAQ_C-PAQ_C_Total',
       'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 
       'PreInt_EduHx-computerinternet_hoursday']

df=train_imp_MICE[features]

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

In [108]:
#New packages needed.

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.datasets import make_regression

In [109]:
#IterativeImputer has a bunch of options, including what type of regression is used for the imputation. Here, I've just gone with the default.

imputer = IterativeImputer(max_iter=10, random_state=497)

df2= imputer.fit_transform(df)

df3 = pd.DataFrame(df2, columns=features)

In [110]:
#Now I fill in the missing values in train_imp_MICE with the MICE-imputed values. I am still using KNN for the pciats values. 

train_imp_MICE[features]=train_imp_MICE[features].fillna(df3[features])
train_imp_MICE[pciats]=train_imp_KNN[pciats]
train_imp_MICE['PCIAT_Total_Imputed']=train_imp_KNN['PCIAT_Total_Imputed']
train_imp_MICE['sii_Imputed']=train_imp_KNN['sii_Imputed']

#Now I can export to a csv.

train_imp_MICE.to_csv('train_imp_MICE.csv', index=False)

In [111]:
#We have now imputed all missing data except for seasons.
 
train_imp_MICE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 84 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   id                                      3168 non-null   object  
 1   Basic_Demos-Enroll_Season               3168 non-null   object  
 2   Basic_Demos-Age                         3168 non-null   int64   
 3   Basic_Demos-Sex                         3168 non-null   int64   
 4   CGAS-Season                             2065 non-null   object  
 5   CGAS-CGAS_Score                         3168 non-null   float64 
 6   Physical-Season                         2642 non-null   object  
 7   Physical-BMI                            3168 non-null   float64 
 8   Physical-Height                         3168 non-null   float64 
 9   Physical-Weight                         3168 non-null   float64 
 10  Physical-Waist_Circumference            3168 non